In [240]:
! pip install nltk

In [241]:
! pip install openpyxl

In [242]:
! pip install -U scikit-learn

In [243]:
from nltk.metrics.agreement import AnnotationTask
from nltk.metrics import masi_distance
from nltk.metrics.distance import jaccard_distance
import pandas as pd
import os
from sklearn.metrics import jaccard_score
import numpy as np
import string

In [244]:
high_level_mapping = {
    'abbreviation without context':'lack of context',
    'unclear intention': 'lack of context',
    'write up': 'content generation',
    'generate persona': 'content generation',
    'exam reference': 'content generation',
    'generate question': 'content generation',
    'generate code template': 'content generation',
    'code': 'content generation',
    'generate test': 'content generation',
    'elaboration': 'content generation',
    'project assistance': 'content generation',
    'summarise topic': 'summarisation',
    'summarise literature': 'summarisation',
    'definition': 'understand concept',
    'understand': 'understand concept',
    'application': 'understand concept',
    'assistance in using application': 'it assistance',
    'technical documentation': 'it assistance',
    'set up': 'it assistance',
    'debugging': 'programming assistance',
    'code validation': 'programming assistance',
    'code suggestion': 'programming assistance',
    'writing refinement': 'language assistance',
    'sensitive writing': 'language assistance',
    'sutd-based information': 'search engine',
    'retrieve data': 'search engine',
    'find examples': 'search engine',
    'industry knowledge': 'search engine',
    'search engine query': 'search engine',
    'avoid plagiarism detection': 'academic dishonesty',
    'attempt to obtain answer for an assignment verbatim': 'academic dishonesty',
    'possible academic dishonesty': 'academic dishonesty',
    'sanity check': 'testing',
    'testing capabilities': 'testing',
    'gptlearn technical specification': 'clarify',
    'role-playing': 'prompt engineering',
    'establishing context': 'prompt engineering',
    'structured output format': 'prompt engineering',
    'idea validation': 'evaluation'
}

only_level1 = ['non-course related', 'follow up conversation', 'idea generation', 'malformed query', 'correcting model response', 'hallucination', 'oracle']

In [245]:
def modify_list(codes_list, code_dependency):
    codes_list = [item.lower() for item in codes_list if item]
    for code in codes_list:
        if code in code_dependency.values() or code in only_level1:
            pass
        elif code in code_dependency.keys():
            codes_list.append(code_dependency[code])
        else:
            print(f'Error: Unkown code -> {code} in the list {codes_list}')
    return list(set(codes_list))

all_data = []
for file in os.listdir('ENCODED'):
    if file.endswith('.xlsx') and not file.startswith('~'):
        print(file)
        df = pd.read_excel(os.path.join('ENCODED', file))
        df['quotation'] = df['quotation'].str.replace(f'[{string.punctuation}]', '', regex=True)
        df['quotation'] = df['quotation'].str.replace('\n', '')
        df['quotation'] = df['quotation'].str.replace(r'\s+', '', regex=True)
        df['quotation'] = df['quotation'].str.strip()
        df['quotation'] = df['quotation'].str.lower()
        df = df.sort_values(by=['document', 'quotation'])
        # Drop the unnecessary columns
        df = df.drop(columns=['comment'])

        df['codes'] = df['codes'].fillna('')
        df['codes'] = df['codes'].astype(str)
        df['codes'] = df['codes'].str.split(', ')
        df['codes'] = df['codes'].apply(lambda x: modify_list(x, high_level_mapping))
        df['codes'] = df['codes'].apply(sorted)
        all_data.append(df)

GPTLearn (2).xlsx
GPTLearn Coding 1 - Daniel.xlsx
GPTLearn Coding 1 - Ivan.xlsx


In [246]:
merged_df = all_data[0].merge(all_data[1], on=['document', 'quotation'], how='outer', suffixes=('_coder1', '_coder2'))
merged_df = merged_df.merge(all_data[2], on=['document', 'quotation'], how='outer')
merged_df['codes_coder3'] = merged_df['codes']
merged_df = merged_df.drop(columns=['codes'])
merged_df['codes_coder1'] = merged_df['codes_coder1'].fillna('[]').astype(str)
merged_df['codes_coder2'] = merged_df['codes_coder2'].fillna('[]').astype(str)
merged_df['codes_coder3'] = merged_df['codes_coder3'].fillna('[]').astype(str)


In [247]:
merged_df.sort_values(by=['document', 'quotation'])

,document,quotation,codes_coder1,codes_coder2,codes_coder3
0,1.docx,explainwhattheyarethenexplainhtelinktothis,"['understand', 'understand concept']","['understand', 'understand concept']","['understand', 'understand concept']"
1,1.docx,firsthelpmefindconceptsonempathythatarerelevan...,['idea generation'],['idea generation'],['idea generation']
2,1.docx,•empathywalkexperiencedwhatitslikeasapregnantw...,"['establishing context', 'prompt engineering']","['establishing context', 'prompt engineering']","['establishing context', 'prompt engineering']"
3,10.docx,assumethatwehavethefollowingtrainingdataavaila...,"['academic dishonesty', 'attempt to obtain ans...","['academic dishonesty', 'attempt to obtain ans...","['academic dishonesty', 'attempt to obtain ans..."
4,100.docx,iamplanningtouseasimpleframeworksojustthebuilt...,"['establishing context', 'prompt engineering']","['establishing context', 'prompt engineering']","['establishing context', 'prompt engineering']"
...,...,...,...,...,...
1566,996.docx,writeanessaybasedontheprompthowdoyouseethetheo...,"['content generation', 'write up']",['non-course related'],['non-course related']
1567,996.docx,youarea23yearolduniversitystudentwhoattended2y...,"['prompt engineering', 'role-playing']","['prompt engineering', 'role-playing']","['prompt engineering', 'role-playing']"
1568,997.docx,whatbiblecharacteristhisriddledepictingtwiceha...,"['non-course related', 'understand', 'understa...","['understand', 'understand concept']","['testing', 'testing capabilities']"
1569,998.docx,whatbiblecharacteristhisriddledepictingtwiceha...,"['definition', 'non-course related', 'understa...","['understand', 'understand concept']","['testing', 'testing capabilities']"


In [248]:
task_data = []
for idx, row in merged_df.iterrows():
    try:
        for i in range(len(all_data)):
            person = f'codes_coder{i+1}'
            task_data.append((person, idx, frozenset(row[person])))
    except:
        for i in range(len(all_data)):
            person = f'codes_coder{i+1}'
            print(row[person], end=" ")
        print()
    
# Initialize AnnotationTask with masi_distance
task = AnnotationTask(distance=masi_distance)

# Load the task data
task.load_array(task_data)

# Compute Krippendorff's alpha
alpha = task.alpha()
print(f"Krippendorff's alpha: {alpha}")

Krippendorff's alpha: 0.7457202409183512


In [249]:
def calculate_alpha(df):
    task_data = []
    for idx, row in df.iterrows():
        try:
            for i in range(len(all_data)):
                person = f'codes_coder{i+1}'
                task_data.append((person, idx, frozenset(row[person])))
        except:
            for i in range(len(all_data)):
                person = f'codes_coder{i+1}'
                print(row[person], end=" ")
            print()
        
    # Initialize AnnotationTask with masi_distance
    task = AnnotationTask(distance=masi_distance)

    # Load the task data
    task.load_array(task_data)

    # Compute Krippendorff's alpha
    alpha = task.alpha()
    print(f"Krippendorff's alpha: {alpha}")
    return alpha

In [250]:
criteria = 0.4
selected_rows = []
rejected_rows = []
for idx, row in merged_df.iterrows():

    coder1_labels = frozenset(row['codes_coder1'])
    coder2_labels = frozenset(row['codes_coder2'])
    coder3_labels = frozenset(row['codes_coder3'])

    distance = masi_distance(coder1_labels, coder2_labels)
    if distance < criteria:
        distance = masi_distance(coder1_labels, coder3_labels)
        if distance > criteria:
            distance = masi_distance(coder1_labels, coder3_labels)
            if distance > criteria:
                row['distance'] = distance
                selected_rows.append(row)
                continue
    rejected_rows.append(row)
                
high_distance_df = pd.DataFrame(selected_rows)
low_distance_df = pd.DataFrame(rejected_rows)
assert high_distance_df.shape[0] + low_distance_df.shape[0] == merged_df.shape[0]
calculate_alpha(high_distance_df)
calculate_alpha(low_distance_df)
calculate_alpha(merged_df)

Krippendorff's alpha: 0.4036383546222746
Krippendorff's alpha: 0.7809572906768107
Krippendorff's alpha: 0.7457202409183512


0.7457202409183512

In [251]:
low_distance_df.to_excel("low_distance_df.xlsx")  

In [252]:
high_distance_df.to_excel("high_distance_df.xlsx")  